In [16]:
import numpy as np

In [17]:
GRID_SIZE = 3 * 3
NB_INDIVIDUALS_POPULATION = 50

In [18]:
def generate_individual():
    """
    Initialize a random Sudoku grid where each cell has an equal chance of being a number from 1 to 9.
    So the grid can be invalid. But aftern, we will use a genetic to make the grid valid.

    Returns:
        A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).
    """
    min_value = 1
    max_value = GRID_SIZE + 1
    size = (GRID_SIZE, GRID_SIZE)
    return np.random.randint(min_value, max_value, size=size)


In [19]:
generate_individual()

array([[4, 1, 2, 3, 9, 5, 4, 8, 4],
       [7, 3, 6, 9, 6, 9, 9, 1, 8],
       [3, 7, 8, 4, 2, 1, 3, 6, 4],
       [7, 3, 2, 3, 7, 1, 5, 1, 2],
       [5, 3, 7, 2, 6, 1, 6, 2, 4],
       [9, 1, 6, 5, 4, 9, 2, 5, 8],
       [6, 1, 3, 4, 6, 1, 3, 6, 5],
       [5, 9, 2, 5, 3, 8, 3, 2, 4],
       [4, 5, 3, 1, 9, 2, 2, 8, 7]])

In [20]:
def generate_population():
    """
    Generates a population by creating a list of individuals using the generate_individual function.

    Returns:
        list: A population consisting of multiple individuals.
    """
    return [generate_individual() for x in range(NB_INDIVIDUALS_POPULATION)]

In [33]:
def compute_fitness(individual):
    """
    Computes the fitness score of an individual.

    Args:
        individual (numpy.ndarray): A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).

    Returns:
        int: Fitness score of the individual.
    """
    fitness = 0

    for i in range(GRID_SIZE):
        unique_digit_column = np.unique(i[:, 0])
        unique_digit_line = np.unique(individual[i])
        fitness += GRID_SIZE - len(unique_digit_line)
        fitness += GRID_SIZE - len(unique_digit_column)
        
    return fitness